# Illustration of mix-SQP solver applied to a small data set, and a large one

## Analysis setup

We begin by loading the Distributions and LowRankApprox Julia packages, as well as some function definitions used in the code chunks below.

In [1]:
using Distributions
using LowRankApprox
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/mixSQP.jl");

Next, initialize the sequence of pseudorandom numbers.

In [2]:
srand(1);

## Generate a small data set

Let's start with a smaller example with 50,000 samples.

In [3]:
x = normtmixdatasim(round(Int,5e4));

## Compute the likelihood matrix

Compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals, with $k = 20$. Note that the rows of the likelihood matrix are normalized by default.

In [4]:
sd = autoselectmixsd(x,nv = 20);
L  = normlikmatrix(x,sd = sd);
size(L)

(50000, 20)

## Fit mixture model using SQP algorithm 

First we run the mix-SQP algorithm once to precompile the function.

In [5]:
out = mixSQP(L,verbose = false);

Observe that only a small number of iterations is needed to converge to the solution of the constrained optimization problem.

In [6]:
F = pqrfact(L, rtol = 1e-8);

In [7]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixSQP(L,x = x0,pqrtol = 1e-10);

Running SQP algorithm with the following settings:
- 50000 x 20 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-10
- partial SVD max. error = 1.40e-09
iter       objective -min(g+1) #nnz #qp
   1 3.03733620e+04 +6.30e-01   20   0
   2 2.09533189e+04 +5.80e+04    1  27
   3 1.28052235e+04 +2.01e+04    3  54
   4 1.11157460e+04 +8.72e+03    3  31
   5 1.09368736e+04 +4.16e+03    3   9
   6 1.07219448e+04 +2.01e+03    3  18
   7 1.05948304e+04 +1.03e+03    3  12
   8 1.05171906e+04 +5.08e+02    3  10
   9 1.03014356e+04 +2.50e+02    2  22
  10 1.01823391e+04 +1.28e+02    3  21
  11 1.01284988e+04 +6.46e+01    3  12
  12 1.00402287e+04 +3.20e+01    3   9
  13 9.89728597e+03 +1.61e+01    3  18
  14 9.85080002e+03 +8.00e+00    3   7
  15 9.81499312e+03 +3.85e+00    3   6
  16 9.77433654e+03 +1.81e+00    3  12
  17 9.75246378e+03 +8.28e-01    4  12
  18 9.74082690e+03 +3.51e-01    4   6
  19 9.73160828e+03 +1.06e-01    4  18
 

## Generate a larger data set

Next, let's see what happens when we use the SQP algorithm to fit a mixture model to a much larger data set.

In [8]:
x = normtmixdatasim(round(Int,1e6));

## Compute the likelihood matrix

As before, we compute the $n \times k$ likelihood matrix for a mixture of zero-centered normals. This time, we use a finer grid of $k = 100$ normal densities.

In [9]:
sd = autoselectmixsd(x,nv = 100);
L  = normlikmatrix(x,sd = sd);
size(L)

(1000000, 100)

## Fit mixture model using SQP algorithm 

Even on this much larger data set, only a small number of iterations is needed to compute the solution.

In [10]:
k   = size(L,2);
x0  = ones(k)/k;
out = mixSQP(L,x = x0,pqrtol = 1e-10);

Running SQP algorithm with the following settings:
- 1000000 x 100 data matrix
- convergence tolerance = 1.00e-08
- zero threshold        = 1.00e-03
- partial SVD tolerance  = 1.00e-10
- partial SVD max. error = 3.52e-08
iter       objective -min(g+1) #nnz #qp
   1 7.00907581e+05 +8.02e-01  100   0
   2 4.68009504e+05 +1.87e+04    2 100
   3 2.60982445e+05 +6.57e+03    3 100
   4 2.09623970e+05 +2.85e+03    3  62
   5 2.04474816e+05 +1.36e+03    3  90
   6 2.02491333e+05 +6.70e+02    2  66
   7 2.00599453e+05 +3.30e+02    3  54
   8 1.99393417e+05 +1.64e+02    3  19
   9 1.98446578e+05 +8.16e+01    3  16
  10 1.97713280e+05 +4.05e+01    3  18
  11 1.97150964e+05 +2.01e+01    3  12
  12 1.96760965e+05 +9.98e+00    4  48
  13 1.96488688e+05 +4.92e+00    4  46
  14 1.96252124e+05 +2.38e+00    5  38
  15 1.96075686e+05 +1.11e+00    4  31
  16 1.95940602e+05 +4.85e-01    5  41
  17 1.95857390e+05 +1.82e-01    4  57
  18 1.95813182e+05 +5.15e-02    4  38
  19 1.95799181e+05 +7.26e-03    4  3

## Session information

The section gives information about the computing environment used to generate the results contained in this
notebook, including the version of Julia, and the versions of the Julia packages used here. 

In [11]:
Pkg.status("Distributions");
Pkg.status("LowRankApprox");

 - Distributions                 0.15.0
 - LowRankApprox                 0.1.0


In [12]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-7567U CPU @ 3.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Prescott)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, broadwell)
